In [1]:
import pandas as pd

In [2]:
# Add columns from seed person and houeshold data to final output

In [49]:
df_syn_pop = pd.read_csv(r'C:\Users\bnichols\POPULATIONSIM_2020\populationsim\psrc\output\synthetic_persons.csv')

In [50]:
df_syn_hh = pd.read_csv(r'C:\Users\bnichols\POPULATIONSIM_2020\populationsim\psrc\output\synthetic_households.csv')

In [5]:
df_seed_pop = pd.read_csv(r'R:\e2projects_two\SyntheticPopulation_2018\populationsim_inputs\data\seed_persons.csv')

In [6]:
df_seed_hh = pd.read_csv(r'R:\e2projects_two\SyntheticPopulation_2018\populationsim_inputs\data\seed_households.csv')

In [51]:
len(df_syn_pop)

3995623

In [53]:
# Household Records
df_new_pop = df_syn_pop.merge(df_seed_pop[['ESR','PINCP','SCHL','SCHG','WKHP','RELP','hhnum','SPORDER']],
                left_on=['hh_id','per_num'], right_on=['hhnum','SPORDER'])
df_new_pop.columns = [i.lower() for i in df_new_pop.columns]
# df_new_pop.drop(['new_per_num'], axis=1, inplace=True)

In [54]:
len(df_new_pop)

3995623

In [11]:
# Load adjusted household seeds
# df_seed_hh_adj = pd.read_csv(r'R:\e2projects_two\SyntheticPopulation_2018\results_with_race\adjusted_seed_households.csv')

In [55]:
# Household Records
# df_new_hh = df_syn_hh.merge(df_seed_hh[['hhnum','TEN','PUMA','NP']], left_on=['hh_id','PUMA','NP'], 
#                             right_on=['hhnum','PUMA','NP'], how='left')

# # Drop the NP column and replace with NP_adjusted - this represents the persons added to this household
# df_new_hh['NP'] = df_new_hh['NP_adjusted'].copy()
# df_new_hh.drop(['NP_adjusted','NP_adjusted','hh_id','new_pums_hh_id'], axis=1, inplace=True)
df_new_hh = df_syn_hh.copy()

In [56]:
df_syn_hh.head()

,household_id,PUMA,block_group_id,hh_id,NP,VEH,worker_count,HHT,HINCP,WGTP
0,1,11802,5335814001,68606,1,3.0,1,4.0,45000.0,31
1,2,11802,5335814001,68606,1,3.0,1,4.0,45000.0,31
2,3,11802,5335814001,68606,1,3.0,1,4.0,45000.0,31
3,4,11802,5335814001,68606,1,3.0,1,4.0,45000.0,31
4,5,11802,5335814001,68606,1,3.0,1,4.0,45000.0,31


In [13]:
print(len(df_new_hh))
print(len(df_syn_hh))

4047846
4047846


In [14]:
df_new_hh.head()

,PUMA,block_group_id,household_id,per_num,AGEP,RAC1P,SEX,SCH,is_worker,NP,person_id,clone,hhnum,TEN
0,11802,5335814001,1,1,25,1,1,1.0,1,1,1,0,68606.0,1.0
1,11802,5335814001,2,1,25,1,1,1.0,1,1,2,0,68606.0,1.0
2,11802,5335814001,3,1,25,1,1,1.0,1,1,3,0,68606.0,1.0
3,11802,5335814001,4,1,25,1,1,1.0,1,1,4,0,68606.0,1.0
4,11802,5335814001,5,1,25,1,1,1.0,1,1,5,0,68606.0,1.0


In [15]:
# df_new_hh.to_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\final_output\adjusted_synthetic_households.csv',
#                  index=False)

In [16]:
# df_new_pop.to_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\final_output\adjusted_synthetic_persons.csv',
#                  index=False)

In [17]:
# Update block group ID

In [57]:
block_group_table = pd.read_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\census_block_groups.csv')

In [58]:
# 2+3+6+1=12
# state county tract block group

In [59]:
# Drop leading zeros to match format of synthetic population
block_group_table['state'] = block_group_table['census_2010_block_group_id'].apply(lambda x: int(str(x)[0:2]))
block_group_table['county'] = block_group_table['census_2010_block_group_id'].apply(lambda x: int(str(x)[3:5]))
block_group_table['tract'] = block_group_table['census_2010_block_group_id'].apply(lambda x: int(str(x)[5:11]))
block_group_table['block_group'] = block_group_table['census_2010_block_group_id'].apply(lambda x: int(str(x)[11:]))

block_group_table['newid'] = block_group_table['state'].astype('str') + block_group_table['county'].astype('str') + block_group_table['tract'].astype('str') + block_group_table['block_group'].astype('str')
# block_group_table['newid'] = block_group_table['newid'].astype('int')

In [60]:
df_new_hh['newid'] = df_new_hh['block_group_id'].astype('str')

In [61]:
df = block_group_table.merge(df_new_hh, on='newid')

In [62]:
df.head()

,census_2010_block_group_id,census_block_group_id,census_tract_id,state,county,tract,block_group,newid,household_id,PUMA,block_group_id,hh_id,NP,VEH,worker_count,HHT,HINCP,WGTP
0,530330001001,1137,1,53,33,100,1,53331001,112161,11602,53331001,47010,1,1.0,0,4.0,144000.0,8
1,530330001001,1137,1,53,33,100,1,53331001,112162,11602,53331001,13112,1,1.0,0,4.0,134100.0,7
2,530330001001,1137,1,53,33,100,1,53331001,112163,11602,53331001,70579,1,1.0,0,4.0,126370.0,23
3,530330001001,1137,1,53,33,100,1,53331001,112164,11602,53331001,70579,1,1.0,0,4.0,126370.0,23
4,530330001001,1137,1,53,33,100,1,53331001,112165,11602,53331001,47010,1,1.0,0,4.0,144000.0,8


In [63]:
len(df_new_hh)

1605216

In [64]:
df.to_csv(r'C:\Users\bnichols\POPULATIONSIM_2020\populationsim\psrc\output\final_adjusted_synthetic_households.csv')

In [65]:
df_new_pop['newid'] = df_new_pop['block_group_id'].astype('str')
df_person = block_group_table.merge(df_new_pop, on='newid')

In [66]:
df_person.to_csv(r'C:\Users\bnichols\POPULATIONSIM_2020\populationsim\psrc\output\final_adjusted_synthetic_persons.csv',
                index=False)

In [ ]:
# test = pd.read_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\final_output\adjusted_synthetic_persons.csv')

In [67]:
len(df_person)

3995623

In [68]:
len(df)

1605216

In [69]:
df.head()

,census_2010_block_group_id,census_block_group_id,census_tract_id,state,county,tract,block_group,newid,household_id,PUMA,block_group_id,hh_id,NP,VEH,worker_count,HHT,HINCP,WGTP
0,530330001001,1137,1,53,33,100,1,53331001,112161,11602,53331001,47010,1,1.0,0,4.0,144000.0,8
1,530330001001,1137,1,53,33,100,1,53331001,112162,11602,53331001,13112,1,1.0,0,4.0,134100.0,7
2,530330001001,1137,1,53,33,100,1,53331001,112163,11602,53331001,70579,1,1.0,0,4.0,126370.0,23
3,530330001001,1137,1,53,33,100,1,53331001,112164,11602,53331001,70579,1,1.0,0,4.0,126370.0,23
4,530330001001,1137,1,53,33,100,1,53331001,112165,11602,53331001,47010,1,1.0,0,4.0,144000.0,8


In [71]:
df_person.columns

Index([u'census_2010_block_group_id', u'census_block_group_id',
       u'census_tract_id', u'state', u'county', u'tract', u'block_group',
       u'newid', u'puma', u'block_group_id', u'hh_id', u'household_id',
       u'per_num', u'agep', u'rac1p', u'sex', u'sch', u'is_worker', u'esr',
       u'pincp', u'schl', u'schg', u'wkhp', u'relp', u'hhnum', u'sporder'],
      dtype='object')

In [43]:
np.hstack(mylist + [[3,3,4]])

array([1, 2, 3, 1, 2, 3, 3, 3, 4])

SyntaxError: invalid syntax (<ipython-input-40-e730c4aaa240>, line 1)